In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import datetime
import sys
import re
import gc
import glob

import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from utils import logger_func, get_categorical_features, get_numeric_features, pararell_process
logger = logger_func()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)

key = 'qid'
qt = 'question_text'
seed = 1208

2018-11-17 11:06:45,902 utils 353 [INFO]    [logger_func] start 


In [2]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

### add for TensorBoard
import keras.callbacks
import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
print(tf.test.is_built_with_cuda())

old_session = KTF.get_session()

session = tf.Session('')
KTF.set_session(session)
KTF.set_learning_phase(1)
### 

Using TensorFlow backend.


True


In [4]:
%%time
# train = utils.read_pkl_gzip(path='../input/1116_train_wordnet_lemma_dict.gz')
# train_df = utils.read_df_pkl('../input/train*.p')[['qid', 'target']]
# tmp = train.copy()
tmp_train = utils.read_df_pkl('../input/wn_bagging_train*.p')

# # for uid, value in tmp.items():
# def pararell_val_join(args):
#     uid = args[0]
#     value = args[1]
#     df_dict = {}
#     np.random.seed(seed)
#     val_len = int(len(value)*0.5)
#     try:
#         value = np.random.choice(a=value, size=val_len)
#     except ValueError:
#         pass
#     df_dict[uid] = " ".join(value)
#     return df_dict
# tmp_dict = {}
# p_list = pararell_process(pararell_val_join, tmp.items())
# [tmp_dict.update(p) for p in p_list]
# tmp_train = pd.Series(tmp_dict).to_frame()
# tmp_train = tmp_train.join(train_df.set_index('qid'))
# tmp_train.rename(columns={0:qt}, inplace=True)

## split to train and val
train_df, val_df = train_test_split(tmp_train, test_size=0.2, random_state=seed)

## some config values 
embed_size = 300 # how big is each word vector
# Current Best 30000
max_features = 30000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a question to use

## fill up the missing values
train_X = train_df["question_text"].fillna("_na_").values
val_X = val_df["question_text"].fillna("_na_").values
# test_X = test_df["question_text"].fillna("_na_").values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
# test_X = tokenizer.texts_to_sequences(test_X)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
# test_X = pad_sequences(test_X, maxlen=maxlen)

## Get the target values
train_y = train_df['target'].values
val_y = val_df['target'].values

100%|██████████| 3/3 [00:00<00:00,  3.44it/s]


CPU times: user 2min 12s, sys: 1.12 s, total: 2min 13s
Wall time: 2min 13s


In [24]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size)(inp)
# x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 100, 300)          6000000   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 100, 128)          187392    
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 16)                2064      
_________________________________________________________________
dropout_6 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 17        
Total para

In [25]:
## Train the model 
tb_cb = keras.callbacks.TensorBoard(log_dir="../tflog/", histogram_freq=1, write_grads=True, write_graph=True, write_images=True)
model.fit(train_X, train_y, batch_size=512, epochs=2,
          ## add 1 line
#           callbacks=[tb_cb],
          validation_data=(val_X, val_y)
         )

### add for TensorBoard
KTF.set_session(old_session)
###

Train on 1044897 samples, validate on 261225 samples
Epoch 1/2
1044897/1044897 [==============================] - 53s 51us/step - loss: 0.1665 - acc: 0.9429 - val_loss: 0.1493 - val_acc: 0.9456
Epoch 2/2
1044897/1044897 [==============================] - 51s 49us/step - loss: 0.1423 - acc: 0.9478 - val_loss: 0.1488 - val_acc: 0.9456


In [26]:
pred_noemb_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_noemb_val_y>thresh).astype(int))))
# del model, inp, x
# import gc; gc.collect()
# time.sleep(10)

261225/261225 [==============================] - 4s 14us/step
F1 score at threshold 0.1 is 0.4135253040157242
F1 score at threshold 0.11 is 0.4262421320476764
F1 score at threshold 0.12 is 0.43750873759261844
F1 score at threshold 0.13 is 0.44815238926150097
F1 score at threshold 0.14 is 0.4574009598193281
F1 score at threshold 0.15 is 0.4658498638662
F1 score at threshold 0.16 is 0.4732842793271448
F1 score at threshold 0.17 is 0.4797539934783506
F1 score at threshold 0.18 is 0.48527384915502214
F1 score at threshold 0.19 is 0.49118607181719254
F1 score at threshold 0.2 is 0.4962114458905332
F1 score at threshold 0.21 is 0.5014047189420068
F1 score at threshold 0.22 is 0.5064865243916692
F1 score at threshold 0.23 is 0.5104067214053848
F1 score at threshold 0.24 is 0.513340318218367
F1 score at threshold 0.25 is 0.515124422589778
F1 score at threshold 0.26 is 0.5171986756640634
F1 score at threshold 0.27 is 0.5182937803717893
F1 score at threshold 0.28 is 0.5209701473012218
F1 score a

In [5]:
utils.start()
EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
        
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
utils.end()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 300)          9000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 128)          140544    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total para

In [6]:
utils.start()
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))
utils.end()

Train on 1044897 samples, validate on 261225 samples
Epoch 1/2
1044897/1044897 [==============================] - 55s 52us/step - loss: 0.1569 - acc: 0.9444 - val_loss: 0.1464 - val_acc: 0.9465
Epoch 2/2
1044897/1044897 [==============================] - 52s 50us/step - loss: 0.1385 - acc: 0.9481 - val_loss: 0.1462 - val_acc: 0.9469


In [7]:
pred_glove_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_glove_val_y>thresh).astype(int))))

261225/261225 [==============================] - 3s 12us/step
F1 score at threshold 0.1 is 0.4836587701403056
F1 score at threshold 0.11 is 0.4921298200792431
F1 score at threshold 0.12 is 0.49971978748683005
F1 score at threshold 0.13 is 0.5064435379195253
F1 score at threshold 0.14 is 0.5119274809160306
F1 score at threshold 0.15 is 0.5158870255957635
F1 score at threshold 0.16 is 0.5197421434327155
F1 score at threshold 0.17 is 0.5233119178470985
F1 score at threshold 0.18 is 0.5260098808485906
F1 score at threshold 0.19 is 0.5271225778593405
F1 score at threshold 0.2 is 0.5296788574259061
F1 score at threshold 0.21 is 0.5306179696485175
F1 score at threshold 0.22 is 0.5303883189987714
F1 score at threshold 0.23 is 0.5311681508643269
F1 score at threshold 0.24 is 0.5314081131461372
F1 score at threshold 0.25 is 0.530355854772782
F1 score at threshold 0.26 is 0.5300137467542386
F1 score at threshold 0.27 is 0.5282445588144107
F1 score at threshold 0.28 is 0.5259681021737078
F1 score 